In [1]:
import numpy as np
import pandas as pd
import random
import scipy.optimize
import itertools

In [2]:
def m1(M,q):
    comp_m1 = M/(1+q)
    return comp_m1

def m2(M,q):
    comp_m2 = q*M/(1+q)
    return comp_m2

def M(m1,m2):
    return m1+m2

def v_crit(m11,m12,m21,m22,a1,a2):
    # G=1
    return np.sqrt(((M(m11,m12)+M(m21,m22)) / (M(m11,m12)*M(m21,m22))) * ((m11*m12/a1)+(m21*m22/a2)))

def b_max(v,a1):
    return (4./v + 3)*a1   # note: v is v/v_crit

In [7]:
# model system, and grid we will use for varying parameters
mdl_sys={}
mdl_sys['v'] = list([0.01])    # v_inf/v_crit
mdl_sys['m'] = list([20.0])   # Msun
mdl_sys['a'] = list([1.0])   # AU
mdl_sys['e'] = list([0.000001])
mdl_sys['alpha'] = list([1.0])
mdl_sys['q'] = list([1.0])

# grids
gs = 50   # grid size
alpha_grid = np.logspace(-2,2,gs)
e1_grid = np.linspace(0.00001,0.98,gs)
e2_grid = np.linspace(0.00001,0.98,gs)
q_grid = np.logspace(np.log10(0.03),np.log10(40),gs)
v_grid = np.logspace(-2,2,gs)

# specify which parameter we want to vary
alpha = alpha_grid# mdl_sys['alpha']
e1 = mdl_sys['e']
e2 = mdl_sys['e']
q = mdl_sys['q']
v = mdl_sys['v']

dat = list(itertools.product(alpha, e1, e2, q, v))
dat = np.asarray(dat)
df = pd.DataFrame(dat,columns=['alpha','e1','e2','q','v_inf/v_crit'])

In [8]:
# 4-body
# 0:M, 1:q, 2:a, 3:e, 4:b, 5:v_crit
grid_name = 'grid_alpha_20M_1AU_0.01V'
grid_path = 'grid_files/'+grid_name+'.dat'
gridinfo_path = 'grid_files/'+grid_name+'_info.txt'

f = open(gridinfo_path, 'w')
f.write('MODEL SYSTEM: m: %f Msun, alpha: %f, a: %f AU, e: %f, q: %f, v: %f v_inf/v_crit \n' % \
        (mdl_sys['m'][0], mdl_sys['alpha'][0], mdl_sys['a'][0], mdl_sys['e'][0], mdl_sys['q'][0], mdl_sys['v'][0]))
f.write('GRID: alpha: %f - %f, e1: %f - %f, e2: %f - %f, q: %f - %f, v: %f - %f \n' % \
            (min(alpha),max(alpha),min(e1),max(e1),min(e2),max(e2),min(q),max(q),min(v),max(v)))
f.close()

f = open(grid_path, 'w')
f.write('1:b/(a1+a2) 2:v/v_crit 3:m11(msun) 4:m12 5:a1(AU) 6:e1 7:m21 8:m22 9:a2 10:e2 11:n_exchange 12:n_s 13:n_b 14:n_b 15:t 16:r')
f.close()

for i in xrange(len(dat)):
    a1 = mdl_sys['a'][0]
    a2 = df['alpha'].iloc[i]*mdl_sys['a'][0]
    e1 = df['e1'].iloc[i]
    e2 = df['e1'].iloc[i]
    m11 = mdl_sys['m'][0]*df['q'].iloc[i]
    m12 = mdl_sys['m'][0]
    m21 = mdl_sys['m'][0]
    m22 = mdl_sys['m'][0]
    v_vcrit = df['v_inf/v_crit'].iloc[i]
    b = b_max(v_vcrit,a1)
    f = open(grid_path, 'a')
    f.write('\n%f %f %f %f %f %f %f %f %f %f 0 0 0 0 0 0' % \
            (b/(a1+a2),v_vcrit,m11,m12,a1,e1,m21,m22,a2,e2))
    f.close()
    

In [91]:
# 3-body   
# 0:M, 1:q, 2:a, 3:e, 4:b, 5:v_crit
grid_name = 'grid_v_samsing'
grid_path = 'grid_files/'+grid_name+'.dat'
gridinfo_path = 'grid_files/'+grid_name+'_info.txt'

f = open(gridinfo_path, 'w')
f.write('MODEL SYSTEM: m: %f, alpha: %f, a: %f, e: %f, q: %f, v: %f \n' % \
        (mdl_sys['m'][0], mdl_sys['alpha'][0], mdl_sys['a'][0], mdl_sys['e'][0], mdl_sys['q'][0], mdl_sys['v'][0]))
f.write('GRID: alpha: %f - %f, e1: %f - %f, e2: %f - %f, q: %f - %f, v: %f - %f \n' % \
            (min(alpha),max(alpha),min(e1),max(e1),min(e2),max(e2),min(q),max(q),min(v),max(v)))
f.close()

f = open(grid_path, 'w')
f.write('1:b/a 2:v/v_crit 3:m1(msun) 4:m2 5:a(AU) 6:e 7:m_s 8:n_exchange 9:n_s 10:n_b 11:n_b 12:t 13:r')
f.close()

for i in xrange(len(dat)):
    a = df['alpha'].iloc[i]*mdl_sys['a'][0]
    e = df['e1'].iloc[i]
    m11 = mdl_sys['m'][0]*df['q'].iloc[i]
    m12 = mdl_sys['m'][0]
    m21 = mdl_sys['m'][0]
    v_vcrit = df['v_inf/v_crit'].iloc[i]
    b = b_max(v_vcrit,a)
    f = open(grid_path, 'a')
    f.write('\n%f %f %f %f %f %f %f 0 0 0 0 0 0' % \
            (b/a,v_vcrit,m11,m12,a,e,m21))
    f.close()

TypeError: 'numpy.float64' object is not iterable